## Lista 1
### Zadanie 1
* Wyznaczymy *macheps* iteracyjnie poprzez dzielenie liczby 1.0 (w różnych reprezentacjach) przez 2, tak długo jak suma 1 + *macheps* będzie > 1.

In [9]:
function print_macheps(breed) # in this and next functions I use breed as Type argument
    macheps = breed(1.0) 
    previous_macheps = macheps
    
    while 1 + macheps != 1 #adding and dividing any Float type T with Int returns T.
        previous_macheps = macheps
        macheps /= 2 
        #println(macheps)
        #println(bitstring(macheps)
    end
    println(breed, ": ", previous_macheps)
end

print_macheps (generic function with 1 method)

In [10]:
println("by me: ")
half = Float16
print_macheps(half)
single = Float32
print_macheps(single)
double = Float64
print_macheps(double)
#####################
println("by Julia: ")
println(half, ": ", eps(half))
println(single, ": ", eps(single))
println(double, ": ", eps(double))

by me: 
Float16: 0.000977
Float32: 1.1920929e-7
Float64: 2.220446049250313e-16
by Julia: 
Float16: 0.000977
Float32: 1.1920929e-7
Float64: 2.220446049250313e-16


Wyniki zgadzają się. Zobaczmy jeszcze *float.h* w C. *float.h* oferuje implemntacje **single** i **double**, nie ma **half**, ale ma za to **long double**. Kod w C oraz wynik:

    #include <stdio.h>
    #include <float.h>

    int main () {

       printf("The macheps value of Float32 = %.10e\n", FLT_EPSILON);
   
       printf("The macheps value of Float64 = %.10e\n", DBL_EPSILON);
   
    }

The macheps value of Float32 = 1.1920928955e-07

The macheps value of Float64 = 2.2204460493e-16


* Wyznaczymy *ete* iteracyjnie poprzez dzielenie liczby 1.0 (w różnych reprezentacjach) przez 2, tak długo jak będzie więszka od 0.0.

In [11]:
function print_eta(breed)
    eta = breed(1.0) 
    previous_eta = eta
    
    while eta > breed(0.0)
        previous_eta = eta
        eta /= 2 
        #println(eta)
        #println(bitstring(eta)
    end
    println(breed, ": ", previous_eta)
end

print_eta (generic function with 1 method)

In [12]:
println("by me: ")
half = Float16
print_eta(half)
single = Float32
print_eta(single)
double = Float64
print_eta(double)
#####################
println("by Julia: ")
println(half, ": ", nextfloat(half(0.0)))
println(single, ": ", nextfloat(single(0.0)))
println(double, ": ", nextfloat(double(0.0)))

by me: 
Float16: 6.0e-8
Float32: 1.0e-45
Float64: 5.0e-324
by Julia: 
Float16: 6.0e-8
Float32: 1.0e-45
Float64: 5.0e-324


* Odpowiedzi na pytania:

Liczba *macheps* jest najmniejszą liczbą maszynową, większą od ϵ gdzie 1 + ϵ = 1. *Macheps* jest więskzy od ϵ o ostani bit reprezentacji, tym samy *macheps* = 2ϵ.

Liczba *eta* jest górną granicą zbioru liczb większych od 0, które nie mają reprezentacji w zbiorze liczb maszynowych, co jest równe $MIN_{sub}$.

Funckje floatmin(Float32), floatmin(Float64) zwracają najmniejszą, znormalizowaną, dodatnią liczbę reprezentowaną przez dany typ zmiennoprzecinkowy (co równa się $MIN_{nor}$). Kiedy nextfloat() zwraca zdenormalizowaną liczbę co pokazuje poniższy kod. W dokumetnacji IEEE-754 możemy przeczytać:
>Underflow:
If the exponent has minimum value (all zero), special rules for denormalized values are followed. The exponent value is set to 2-126 and the "invisible" leading bit for the mantissa is no longer used.
The range of the mantissa is now [0:1).

In [13]:
println(floatmin(Float16))
println(bitstring(floatmin(Float16)))

6.104e-5
0000010000000000


In [14]:
println(nextfloat(Float16(0.0)))
println(bitstring(nextfloat(Float16(0.0))))

6.0e-8
0000000000000001


* Wyznaczymy *max* iteracyjnie poprzez mnożenie x = 1.0 (w różnych reprezentacjach) przez 2, tak długo aż otrzymamy *inf*, wtedy bierzemy poprzednią wartość x i potrzebujemy dodać (zapełnić) pozostałe bity mantysy x, najszybciej osiągniemy to odejmując od x eps(x) co da nam największa liczbę mniejszą od x, a następnie dodając ją do x. Musimy wykonać tę operację ponieważ ciągłe mnożenie przez jeden zapełnia wszytskie bity cechy co powoduje, że x ląduje w special case, mianowicie inf.

In [15]:
function print_max(breed)
    max = breed(1.0) 
    previous_max = max
    
    while isinf(max) == false
        previous_max = max
        max *= 2
        #println(max)
        #println(bitstring(max)
    end

    max = previous_max + (previous_max - eps(previous_max)) # very important bracket
    println(breed, ": ", max)
end

print_max (generic function with 1 method)

In [16]:
println("by me: ")
half = Float16
print_max(half)
single = Float32
print_max(single)
double = Float64
print_max(double)
#####################
println("by Julia: ")
println(half, ": ", floatmax(half))
println(single, ": ", floatmax(single))
println(double, ": ", floatmax(double))

by me: 
Float16: 6.55e4
Float32: 3.4028235e38
Float64: 1.7976931348623157e308
by Julia: 
Float16: 6.55e4
Float32: 3.4028235e38
Float64: 1.7976931348623157e308


In [17]:
println(bitstring(Float16(3.277e4))) # x
println(bitstring((Float16(3.277e4)) - eps(Float16(3.277e4)))) # x - eps(x)
println(bitstring((Float16(3.277e4) + ((Float16(3.277e4)) - eps(Float16(3.277e4)))))) # x + (x - eps(x)) = max
println(bitstring(floatmax(Float16))) # max

0111100000000000
0111011111111110
0111101111111111
0111101111111111


### Zadanie 2
* Będziemy obliczać wyrażenie $3(\frac{4}{3} - 1) - 1$ dla wszystkich typów zmiennoprzecinkowych. Z nadzieją, że otrzymamy *macheps*.

In [18]:
println("by Kahan: ")
println(Float16(3) * ( Float16(4/3) - Float16(1) ) - Float16(1))
println(Float32(3) * ( Float32(4/3) - Float32(1) ) - Float32(1))
println(Float64(3) * ( Float64(4/3) - Float64(1) ) - Float64(1))
println("by Julia: ")
println(eps(Float16))
println(eps(Float32))
println(eps(Float64))

by Kahan: 
-0.000977
1.1920929e-7
-2.220446049250313e-16
by Julia: 
0.000977
1.1920929e-7
2.220446049250313e-16


Z dokładnością do wartości bezwględnej twierdzenie Kahana jest prawdziwe.

### Zadanie 3
Pokażemy rozmieszczenie liczb maszynowych przy pomocy reprezentacji bitowej. Reprezentacje bitowe kolejncyh granic przedziałów z treści zadania w IEE-754 (double):

In [19]:
println("1 bitowo: ", bitstring(1.0))
println("2 bitowo: ", bitstring(2.0))

1 bitowo: 0011111111110000000000000000000000000000000000000000000000000000
2 bitowo: 0100000000000000000000000000000000000000000000000000000000000000


Mamy 52 bity mantysy co oznacza, że za ich pomocą możemy zapisać $2^{52}$ różnych liczb z przedziału [1,2).
Przy stałej cesze kolejną liczbę otrzymujemy poprzez dodanie bita na samym końcu (czyli 52-ego bita), co odpowiada dodaniu $2^{-52}$. Dla dowolnej liczby, z tego przedziału, kolejną otrzymujemy poprzez dodanie do niej stałej wartosci $2^{-52}$, co za tym idzie liczby maszynowe w przedziale [1,2] są rozmieszcozne równomiernie.

In [20]:
println("1/2 bitowo: ", bitstring(0.5))
println("1   bitowo: ", bitstring(1.0))
println("2   bitowo: ", bitstring(2.0))
println("4   bitowo: ", bitstring(4.0))

1/2 bitowo: 0011111111100000000000000000000000000000000000000000000000000000
1   bitowo: 0011111111110000000000000000000000000000000000000000000000000000
2   bitowo: 0100000000000000000000000000000000000000000000000000000000000000
4   bitowo: 0100000000010000000000000000000000000000000000000000000000000000


Dla każdego przedziału $[2^{c}, 2^{c+1}]$ gdzie $c\in\{-1022, -1021, ... ,1023\}$ rozkład kolejncyh liczb maszynowych jest stały, ponieważ pomiędzy kolejnymi liczbami różnica w wartosci mantysy jest stała ( i wynosi $2^{-52}$), natomiast poprzez sposób liczenia wartości liczby w IEEE-754, to jest $s* 2^{c} * m_t$ róznice w wartosćiach liczb maszynowych rosną wraz ze wzrostem cechy. 

### Zadanie 4
* Postaramy się rozwiązać a) i b) za jednym razem. Zaczniemy od 0.0 + $macheps$ i będziemy brali kolejne liczby mające reprezentacje we Float64.

In [21]:
x = Float64(1.0)
while Float64(x * Float64(Float64(1.0)/x)) == Float64(1.0) # Float64 in case this will be run on a computer with 32 bit architecture
    x = nextfloat(x)
end

println(x)

1.000000057228997


W przypadku gdyby górna komórka liczyłą sie zbyt długo, niżej jest wynik i dowód, że nie spełnia tej równości.

In [22]:
1.000000057228997*(1/1.000000057228997)


0.9999999999999999

### Zadanie 5
* Zaimplementujmy 4 metody liczenia iloczynu skalarnego.

In [23]:
function front_sum(A, B, size, breed) # frontward
    sum = breed(0.0)
    for i in 1:size
        sum += breed(A[i]) * breed(B[i])
    end
    return sum
end

function back_sum(A, B, size, breed) # backward
    sum = breed(0.0)
    for i in size:-1:1
        sum += breed(A[i]) * breed(B[i])
    end
    return sum
end

function sum_helper(NewA, breed) # help function, beacuse I am not sure how does Base.sum work
    sum = breed(0.0)
    for i in 1:size(NewA)[1]
        sum += NewA[i]
    end
    return sum
end

function smallest_to_biggest_sum(A, B, size, breed) 
    P = []
    N = []
    for i in 1:size
        mult = breed(A[i]) * breed(B[i]) # simulated dot product
        if mult >= 0 # positive mults go to positive array, same with negative, they go to negative
            append!(P, mult)
        else 
            append!(N, mult)
        end
    end
    P = sort(P, rev = true)
    N = sort(N, rev = true)
    positive = sum_helper(P, breed)
    negative = sum_helper(N, breed)
    return breed(positive + negative)
end

function biggest_to_smallest_sum(A, B, size, breed)
    P = []
    N = []
    for i in 1:size
        mult = breed(A[i]) * breed(B[i]) # simulated dot product
        if mult >= 0 # positive mults go to positive array, same with negative, they go to negative
            append!(P, mult)
        else 
            append!(N, mult)
        end
    end
    P = sort(P)
    N = sort(N)
    positive = sum_helper(P, breed)
    negative = sum_helper(N, breed)
    return breed(positive + negative)
end

biggest_to_smallest_sum (generic function with 1 method)

In [24]:
using LinearAlgebra
A = [2.718281828, -3.141592654, 1.414213562, 0.5772156649, 0.3010299957]
B = [1486.2497, 878366.9879, -22.37492, 4773714.647, 0.000185049]

println("by LinearAlgebra: ")
println(dot(A,B))
println("frontward sum:")
println(front_sum(A,B,5, Float32))
println(front_sum(A,B,5, Float64))
# println(front_sum(A,B,5, BigFloat))
println("backward sum:")
println(back_sum(A,B,5, Float32))
println(back_sum(A,B,5, Float64))
# println(back_sum(A,B,5, BigFloat))
println("biggest to smallest sum:")
println(biggest_to_smallest_sum(A,B,5, Float32))
println(biggest_to_smallest_sum(A,B,5, Float64))
println("smallest to biggest sum:")
println(smallest_to_biggest_sum(A,B,5, Float32))
println(smallest_to_biggest_sum(A,B,5, Float64))

by LinearAlgebra: 
1.0251881368296672e-10
frontward sum:
-0.4999443
1.0251881368296672e-10
backward sum:
-0.4543457
-1.5643308870494366e-10
biggest to smallest sum:
-0.5
0.0
smallest to biggest sum:
-0.5
0.0


Jak widać wyniki bardzo się różnią. Metoda liczenia a) jest zgodna z metodą oferowaną przez pakiet LinearAlgebra natomiast wciąż różny od prawidłowej wartości, to jest $-1.00657107000000 * 10^{-11}$. Jeżeli chodzi o sumy c) i d). Wyniki są identyczne, co jest zrozumiałe bo kolejność wykonywania operacji ma znaczenie gdy liczby są sobie bliskie, natomiast w tym przypadku liczby były daleko od siebie, natomiast wartość ich sumy jest bardzo odległa od prawdziwej. Przykład jest tak dobrany aby dodnie sumy częściowe i ujemne były dużymi bliskimi sobie liczbami o przeciwnych znakach. Po zsumowaniu dostajemy 0.0.

### Zadanie 6

In [25]:
f(x) = sqrt(x^2 + 1.0) - 1.0
g(x) = x^2/(sqrt(x^2 + 1) + 1)

g (generic function with 1 method)

In [26]:
for i in 1:15
    x = 8.0^(-i)
    println("f(8^(-", i, ") = ", f(x))
    println("g(8^(-", i, ") = ", g(x), "\n")
end

f(8^(-1) = 0.0077822185373186414
g(8^(-1) = 0.0077822185373187065

f(8^(-2) = 0.00012206286282867573
g(8^(-2) = 0.00012206286282875901

f(8^(-3) = 1.9073468138230965e-6
g(8^(-3) = 1.907346813826566e-6

f(8^(-4) = 2.9802321943606103e-8
g(8^(-4) = 2.9802321943606116e-8

f(8^(-5) = 4.656612873077393e-10
g(8^(-5) = 4.6566128719931904e-10

f(8^(-6) = 7.275957614183426e-12
g(8^(-6) = 7.275957614156956e-12

f(8^(-7) = 1.1368683772161603e-13
g(8^(-7) = 1.1368683772160957e-13

f(8^(-8) = 1.7763568394002505e-15
g(8^(-8) = 1.7763568394002489e-15

f(8^(-9) = 0.0
g(8^(-9) = 2.7755575615628914e-17

f(8^(-10) = 0.0
g(8^(-10) = 4.336808689942018e-19

f(8^(-11) = 0.0
g(8^(-11) = 6.776263578034403e-21

f(8^(-12) = 0.0
g(8^(-12) = 1.0587911840678754e-22

f(8^(-13) = 0.0
g(8^(-13) = 1.6543612251060553e-24

f(8^(-14) = 0.0
g(8^(-14) = 2.5849394142282115e-26

f(8^(-15) = 0.0
g(8^(-15) = 4.0389678347315804e-28



Funckja $g$ daje lepsze wyniki, co widać już przy $8^{-9}$. Powodem tego zachowania jest pojawienie się operacji odejmowania w funckji $f$. W przypadku funkcji $f$ następuje redukacja cyfr znaczących co było omówione na wykładzie.

### Zadanie 7
Będziemy liczyć:
* przybliżoną wartość pochodnej funckji $f(x) = sin(x) + cos(3x)$ w punkcie $x_0 = 1$, za pomocą przybliżenia $f'(x_0) \approx \tilde{f'}(x_0) = \frac{f(x_0 + h) - f(x_0)}{h}$,

* błąd $|f'(x_0) - \tilde{f'}(x_0)|$,

* oraz $f'(x) = cos(x) - 3sin(3x)$.

In [27]:
f(x) = sin(x) + cos(3x)
f_prim(x) = cos(x) - 3sin(3x)
f_tilda_prim(x, h) = (f(x + h) - f(x))/h

f_tilda_prim (generic function with 1 method)

In [28]:
for n in 0:54
    x_0 = 1.0
    h = 2.0^(-n)
    println("n=", n, " : h=", h, " : 1+h=", x_0 + h, " : ft=", f_tilda_prim(x_0, h), " : err=", abs(f_prim(x_0) - f_tilda_prim(x_0, h)))
end

println("\nactually value of f' in x_0 = ", f_prim(1.0))
    

n=0 : h=1.0 : 1+h=2.0 : ft=2.0179892252685967 : err=1.9010469435800585
n=1 : h=0.5 : 1+h=1.5 : ft=1.8704413979316472 : err=1.753499116243109
n=2 : h=0.25 : 1+h=1.25 : ft=1.1077870952342974 : err=0.9908448135457593
n=3 : h=0.125 : 1+h=1.125 : ft=0.6232412792975817 : err=0.5062989976090435
n=4 : h=0.0625 : 1+h=1.0625 : ft=0.3704000662035192 : err=0.253457784514981
n=5 : h=0.03125 : 1+h=1.03125 : ft=0.24344307439754687 : err=0.1265007927090087
n=6 : h=0.015625 : 1+h=1.015625 : ft=0.18009756330732785 : err=0.0631552816187897
n=7 : h=0.0078125 : 1+h=1.0078125 : ft=0.1484913953710958 : err=0.03154911368255764
n=8 : h=0.00390625 : 1+h=1.00390625 : ft=0.1327091142805159 : err=0.015766832591977753
n=9 : h=0.001953125 : 1+h=1.001953125 : ft=0.1248236929407085 : err=0.007881411252170345
n=10 : h=0.0009765625 : 1+h=1.0009765625 : ft=0.12088247681106168 : err=0.0039401951225235265
n=11 : h=0.00048828125 : 1+h=1.00048828125 : ft=0.11891225046883847 : err=0.001969968780300313
n=12 : h=0.000244140625 

Zmniejszanie wartości $h$ od pewnego momentu nie poprawia wyniku, ponieważ operacja odejmowania na bliskich sobie liczbach redukuje cyfry znaczące. Zmniejszanie $h$ doprowadza do tego, że $x + h \longrightarrow x$ tym samym w liczniku f_tilda_prim() odejmujemy liczby coraz bliższe.

Wartość $1.0 + h$ od $n=53$ jest równa 1.0.

Najlepszy wynik otrzymujemy dla $n = 28$, widać to po najmniejszym błędzie. Może być to odrobine szokujące, ale jest rozsądne, zmniejszanie $h$, czyli tym samym przybliżanie się do rzeczywistej wartości pochodznej w tym punkcie prowadziłoby do lepszego wyniku gdybyśmy nie operowali na liczbach rzeczywistych, niestety mamy wyłacznie liczby maszynowe. 